In [151]:
import os
%load_ext autoreload
%autoreload 2
import pandas as pd
import pyspark
import json
import pyarrow
from pyspark.sql import SparkSession
from pyspark.sql.utils import AnalysisException
from pyspark.sql import Row
from pyspark.sql.window import Window
from py4j.protocol import Py4JJavaError
from datetime import datetime, timedelta

from pyspark.sql.functions import *
from pyspark.sql.types import IntegerType, BooleanType

os.environ['SPARK_HOME'] = 'C:/Users/saul2/Spark_DF/spark-3.5.5-bin-hadoop3'
os.environ['HADOOP_HOME'] = 'C:/Users/saul2/Spark_DF/spark-3.5.5-bin-hadoop3'
os.environ['JAVA_HOME'] = 'C:/Program Files/Java/jdk1.8.0_202'

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
spark = SparkSession.builder.appName("Weather_Session").getOrCreate()

In [11]:
path = "C:/Users/saul2/OneDrive/Desktop/PastProjects/ETL_Project/Final_ETL_Test/table_output/Cloud_Wind_Table.parquet"
try:
    cloud_wind_df = spark.read.parquet(path)
except AnalysisException  as e:
    # Path does not exists
    print(e)

In [13]:
path = "C:/Users/saul2/OneDrive/Desktop/PastProjects/ETL_Project/Final_ETL_Test/table_output/Location_Table.parquet"
try:
    location_df= spark.read.parquet(path)
except AnalysisException  as e:
    # Path does not exists
    print(e)

In [15]:
path = "C:/Users/saul2/OneDrive/Desktop/PastProjects/ETL_Project/Final_ETL_Test/table_output/Sunrise_Sunset_Table.parquet"
try:
    sunrise_sunset_df = spark.read.parquet(path)
except AnalysisException  as e:
    # Path does not exists
    print(e)

In [17]:
path = "C:/Users/saul2/OneDrive/Desktop/PastProjects/ETL_Project/Final_ETL_Test/table_output/Temperature_Table.parquet"
try:
    temperature_df = spark.read.parquet(path)
except AnalysisException  as e:
    # Path does not exists
    print(e)

In [19]:
path = "C:/Users/saul2/OneDrive/Desktop/PastProjects/ETL_Project/Final_ETL_Test/table_output/Weather_Description_Table.parquet"
try:
    weather_description_df = spark.read.parquet(path)
except AnalysisException  as e:
    # Path does not exists
    print(e)

In [127]:
timestamp = datetime.now().strftime("%Y_%m_%d_%H_%M_%S")
print(timestamp)
output_dir = f"table_results_{timestamp}"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

2025_05_18_15_10_28


In [31]:
print("cloud_wind_df")
cloud_wind_df.show(5)
print("location_df")
location_df.show(5)
print("sunrise_sunset_df")
sunrise_sunset_df.show(5)
print("temperature_df")
temperature_df.show(5)
print("weather_description_df")
weather_description_df.show(5)

cloud_wind_df
+-------+---------------------+---------------------+----------+----------+
|City_ID|Cloudiness_Percentage|Wind_Direction_Degree|Gust_Speed|Wind_Speed|
+-------+---------------------+---------------------+----------+----------+
|  98182|                   12|                  339|     14.65|      7.05|
| 108410|                    0|                  327|      5.83|      4.33|
| 112931|                    0|                   68|      1.53|      1.59|
| 184745|                   53|                   77|      1.01|      0.37|
| 264371|                    0|                  306|      6.71|      4.02|
+-------+---------------------+---------------------+----------+----------+
only showing top 5 rows

location_df
+------+-------+-------+--------+---------+
|    ID|   City|Country|Latitude|Longitude|
+------+-------+-------+--------+---------+
| 98182|Baghdad|     IQ| 33.3406|  44.4009|
|108410| Riyadh|     SA| 24.6877|  46.7219|
|112931| Tehran|     IR| 35.6944|  51.4215|
|

In [167]:
# Which cities have the longest daylight duration?
# Convert the time into readable time. Order by daylight hour
top_10_cities = sunrise_sunset_df.select(
    col("City_ID"), 
    date_format(from_unixtime(col("Sunrise") + col("Timezone")),"HH:mm:ss").alias("Sunrise"),
    date_format(from_unixtime(col("Sunset") + col("Timezone")),"HH:mm:ss").alias("Sunset"), 
    round(((col("Sunset") - col("Sunrise")) / 3600),2).alias("Daylight_Hours"),
    col("Timezone")).orderBy(col("Daylight_Hours").desc()).limit(10)

final_table = top_10_cities.join(location_df, top_10_cities["City_ID"] == location_df["ID"]
                ).select(location_df["ID"], 
                         location_df["City"], 
                         location_df["Country"], 
                         top_10_cities["Sunrise"], 
                         top_10_cities["Sunset"], 
                         top_10_cities["Daylight_Hours"], 
                         top_10_cities["Timezone"]
                ).orderBy(col("Daylight_Hours").desc())
final_table.show()

final_table.createOrReplaceTempView("Final_Table")

query = '''
        SELECT ID, CITY, COUNTRY, SUNRISE, SUNSET, DAYLIGHT_HOURS, TIMEZONE, RANK()OVER(ORDER BY DAYLIGHT_HOURS DESC) RANK
        FROM FINAL_TABLE
    '''
result = spark.sql(query)
result.show()

pandas_df = result.toPandas()
#pandas_df = final_table.toPandas()
path = os.path.join(output_dir, 'Longest_Daylight_Table.parquet')
pandas_df.to_parquet(path, engine='pyarrow') 
path = os.path.join(output_dir, 'Longest_Daylight_Table.csv')
pandas_df.to_csv(path, index=False)

+-------+----------+-------+--------+--------+--------------+--------+
|     ID|      City|Country| Sunrise|  Sunset|Daylight_Hours|Timezone|
+-------+----------+-------+--------+--------+--------------+--------+
|3413829| Reykjavik|     IS|21:02:45|15:45:31|         18.71|       0|
| 658225|  Helsinki|     FI|21:34:19|14:59:18|         17.42|   10800|
|3143244|      Oslo|     NO|21:32:59|14:54:09|         17.35|    7200|
|2673730| Stockholm|     SE|21:08:06|14:20:29|         17.21|    7200|
| 524901|    Moscow|     RU|21:12:47|13:39:24|         16.44|   10800|
|2618425|Copenhagen|     DK|21:53:17|14:19:18|         16.43|    7200|
|2964574|    Dublin|     IE|22:20:32|14:22:43|         16.04|    3600|
|2950159|    Berlin|     DE|22:05:44|14:00:05|         15.91|    7200|
|2759794| Amsterdam|     NL|22:40:28|14:33:32|         15.88|    7200|
| 756135|    Warsaw|     PL|21:36:41|13:28:19|         15.86|    7200|
+-------+----------+-------+--------+--------+--------------+--------+

+----

In [177]:
# Which city has the highest difference between actual temperature and feels-like temperature?
# temperature_df
top_10_cities = temperature_df.select(
    col("City_ID"), 
    col("Temp"), 
    col("Feels_Like"), 
    round(abs(col("Temp") - col("Feels_Like")),2).alias("Difference")
        ).orderBy(col("Difference").desc()).limit(10)
#top_10_cities.show()

final_top_10_cities_temperature = top_10_cities.join(location_df, top_10_cities["City_ID"] == location_df["ID"]).select(
    top_10_cities["City_ID"], 
    location_df["City"], 
    location_df["Country"], 
    top_10_cities["Temp"], 
    top_10_cities["Feels_Like"], 
    top_10_cities["Difference"]
        ).orderBy(col("Difference").desc())
#final_top_10_cities_temperature.show()

final_top_10_cities_temperature.createOrReplaceTempView("Final_Table")

query = '''
        SELECT CITY_ID, CITY, COUNTRY, TEMP, FEELS_LIKE, DIFFERENCE, RANK()OVER(ORDER BY DIFFERENCE DESC) RANK
        FROM FINAL_TABLE
    '''

final_result = spark.sql(query)

final_result.show()
pandas_df = final_result.toPandas()
path = os.path.join(output_dir, 'Temperature_Difference_Table.parquet')
pandas_df.to_parquet(path, engine='pyarrow') 
path = os.path.join(output_dir, 'Temperature_Difference_Table.csv')
pandas_df.to_csv(path, index=False)

+-------+---------+-------+-----+----------+----------+----+
|CITY_ID|     CITY|COUNTRY| TEMP|FEELS_LIKE|DIFFERENCE|RANK|
+-------+---------+-------+-----+----------+----------+----+
|1609350|  Bangkok|     TH|30.51|     37.51|       7.0|   1|
|1701668|   Manila|     PH|32.69|     39.69|       7.0|   1|
|1880252|Singapore|     SG|31.07|     38.07|       7.0|   1|
|1642911|  Jakarta|     ID|29.67|     34.03|      4.36|   4|
|1275339|   Mumbai|     IN|28.89|     32.49|       3.6|   5|
| 658225| Helsinki|     FI|  8.0|      4.61|      3.39|   6|
|1581130|    Hanoi|     VN|27.33|     30.26|      2.93|   7|
|2673730|Stockholm|     SE| 4.88|      2.17|      2.71|   8|
|5856195| Honolulu|     US|27.38|     30.04|      2.66|   9|
|3067696|   Prague|     CZ| 7.29|      4.66|      2.63|  10|
+-------+---------+-------+-----+----------+----------+----+



In [ ]:
# Upload information to Cloud Service